In [8]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
import os
import json
import gzip
import time
import calendar
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import date
import warnings
warnings.filterwarnings('ignore')
SETS = './sets/' # папка с файлами с наборами

In [9]:
def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')

# Функция для преобразования данных в геодатафрейм
def create_geodataframe(row, geo_column):
    try:
        geo_data = row[geo_column]

        # Проверяем, является ли geo_data NaN
        if pd.isna(geo_data):
            return gpd.GeoDataFrame(geometry=[None])  # Возвращаем None, если geo_data NaN

        geometry_type = geo_data['type']
        
        if geometry_type == 'Point':
            coordinates = geo_data['coordinates']
            geometry = Point(coordinates)
        elif geometry_type == 'Polygon':
            coordinates = geo_data['coordinates'][0]  # Предполагаем, что координаты переданы в виде (объект)
            geometry = Polygon(coordinates)
        elif geometry_type == 'MultiPolygon':
            geometry = MultiPolygon([Polygon(p[0]) for p in geo_data['coordinates']])
        elif geometry_type == 'LineString':
            coordinates = geo_data['coordinates']
            geometry = LineString(coordinates)
        else:
            # Если тип геометрии не распознан, возвращаем None
            geometry = None
    
    except Exception as e:
        print(f"Ошибка: {e}. Значение: {row[geo_column]}")  # Выводим ошибку и значение, вызвавшее ошибку
        geometry = None  # Возвращаем None в случае ошибки

    return gpd.GeoDataFrame(geometry=[geometry])

In [10]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [11]:
ecology = load_dataset('ecology')
exams = load_dataset('exams')

In [ ]:
%%time
ecology['geo'] = ecology.apply(lambda row: create_geodataframe(row, 'air'), axis=1)